## crawling page

### 라이브러리 설치

```
pip install bs4
pip install pymysql
pip install selenium
```

### chromewebdriver 설치

https://sites.google.com/a/chromium.org/chromedriver/downloads

--- 

참고 : https://greeksharifa.github.io/references/2020/10/30/python-selenium-usage/

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import matplotlib.pyplot as plt
import re
import os
import urllib
import numpy as np
import pandas as pd
import pymysql

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
    else: 
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500:
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
        return resp

In [3]:
db_info = pymysql.connect(user='root', passwd='root',host='localhost',port=3306, db='test')
cursor = db_info.cursor()

In [4]:
from selenium import webdriver
driver = webdriver.Chrome('/SSAFY/python_project/chromedriver')
driver.get('https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail&PublishMonth=36&SortOrder=2&Stockstatus=0&CID=1&page=1')

In [46]:
from selenium.common.exceptions import NoSuchElementException
driver.get(url + '4')
ss_book_boxs = driver.find_elements_by_class_name('ss_book_box')

for ss in ss_book_boxs:
    isbn = ss.find_element_by_class_name('Search3_Result_SafeBasketArea').get_attribute('isbn')
    title = ss.find_element_by_css_selector('a[class=bo3] > b').text
    try:
        filepath = ss.find_element_by_class_name('i_cover').get_attribute('src')
    except NoSuchElementException as e:
        continue

In [8]:
from selenium.common.exceptions import NoSuchElementException

url = 'https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail&PublishMonth=36&SortOrder=2&Stockstatus=0&CID=1&page='
sql = "INSERT INTO book (isbn, title, coverFilepath, category_id) VALUES (%s, %s, %s, %s)"
for i in range(1,11):
    driver.get(url + str(i))
    ss_book_boxs = driver.find_elements_by_class_name('ss_book_box')
    
    for ss in ss_book_boxs:
        isbn = ss.find_element_by_class_name('Search3_Result_SafeBasketArea').get_attribute('isbn')
        title = ss.find_element_by_css_selector('a[class=bo3] > b').text
        try:
            filepath = ss.find_element_by_class_name('i_cover').get_attribute('src')
        except NoSuchElementException as e:
            continue
        
        cursor.execute(sql,(isbn, title, filepath, 1))
    time.sleep(2)
    
db_info.commit()

In [10]:
driver.close()

In [9]:
sql = "SELECT * FROM book;"
cursor.execute(sql)
result = cursor.fetchall()
pd.DataFrame(result)

,0,1,2,3,4
0,492,8954682154,작별하지 않는다,https://image.aladin.co.kr/product/27877/5/cov...,1
1,493,K832734163,"재인, 재욱, 재훈 (리커버 에디션)",https://image.aladin.co.kr/product/27802/70/co...,1
2,494,K962733015,달러구트 꿈 백화점 2,https://image.aladin.co.kr/product/27587/47/co...,1
3,495,K082733434,지구 끝의 온실,https://image.aladin.co.kr/product/27692/63/co...,1
4,496,8954681174,밝은 밤,https://image.aladin.co.kr/product/27541/91/co...,1
...,...,...,...,...,...
487,979,K172638609,천재들의 초상,https://image.aladin.co.kr/product/23510/68/co...,1
488,980,K692733221,어둠의 왕관,https://image.aladin.co.kr/product/27644/70/co...,1
489,981,K822733642,꽃잎을 여미다 1~2 세트 - 전2권,https://image.aladin.co.kr/product/27743/2/cov...,1
490,982,K382733119,작은 아씨들,https://image.aladin.co.kr/product/27589/51/co...,1


In [11]:
db_info.close()